In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U spacy
!python -m spacy download ru_core_news_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-12 11:31:57.487342: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 11:31:58.625146: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 69.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha

In [ ]:
!python -m spacy download ru_core_news_md

2023-05-12 11:32:18.343653: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 11:32:19.813036: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/ru_core_news_md-3.5.0/ru_core_news_md-3.5.0-py3-none-any.whl (41.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd
from itertools import groupby
pd.options.display.max_rows = 600
pd.options.display.max_colwidth = 400

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Kursovaya/для курсовой.xlsx')

In [ ]:
df.head()

,Column10
0,"ЕЊmura, Satoshi#:#Kitasato Institute for Life Sciences, Kitasato University, Tokyo"
1,"Еђsi, Attila#:#Hungarian Academy of Sciences, Hungarian Natural History Museum, Research Group for Paleontology, Ludovika tГ©r 2, Budapest 1083, Hungary ,#;#Butler, Richard J.#:#Bayerische Staatssammlung fГјr PalГ¤ontologie und Geologie, Richard-Wagner-StraГџe 10, Munich 80333, Germany ,#;#Weishampel, David B.#:#Center for Functional Anatomy and Evolution, Johns Hopkins University, Baltimore, ..."
2,"Е TERZL, J.#:#Division of Immunology, Institute of Biology, Czechoslovak Academy of Sciences, Prague"
3,"Е PAДЊEK, M.#:#Jeffery Hale's Hospital, Quebec City"
4,"ГђorД‘eviД‡, Luka#:#Department of Chemical and Pharmaceutical Sciences, INSTM UdR Trieste, University of Trieste, Trieste, Italy#|#Simpson Querrey Institute, Northwestern University, Chicago, USA#;#Arcudi, Francesca#:#Department of Chemical and Pharmaceutical Sciences, INSTM UdR Trieste, University of Trieste, Trieste, Italy#|#Department of Chemistry, Northwestern University, Evanston, USA#;#P..."


## spacy

In [ ]:
import ru_core_news_md
nlp = ru_core_news_md.load()

In [ ]:
test_df=df[0:50]

In [ ]:
test_df

In [ ]:
# def cleaning(df):
#     working_df=df
#     rows=[]
#     for i in range(len(working_df)):
#         final_row=''
#         row=df.iloc[i,0].replace('.','').replace('#',',').split(',')

#         while '|' in row:
#             row.remove('|')
#         while ':' in row:
#             row.remove(':')
#         while ';' in row:
#             row.remove(';')
#         for j in row:
#             final_row=final_row+j+','
#         rows.append(final_row)
#     return rows

In [ ]:
def cleaning111(df):
    working_df=df
    rows=[]
    for i in range(len(working_df)):
        final_row=''
        #row=df.iloc[i,0].replace('.','').replace('#',',').split(',')
        row=df.iloc[i,0].replace('#|#',',').split('#')
        while ':' in row:
            row.remove(':')
        while ';' in row:
            row.remove(';')
        #print(row)
        org_row=','.join(map(str, row[1::2])).split(',')
        for j in org_row:
            final_row=final_row+j+','
        rows.append(','.join(map(str, list(filter(None, final_row.split(','))))))
    return rows

In [ ]:
def cleaning(df):
    working_df=df
    rows=[]
    for i in range(len(working_df)):
        row=df.iloc[i,0].replace('#|#',',').split('#')
        while ':' in row:
            row.remove(':')
        while ';' in row:
            row.remove(';')
        org_row=','.join(map(str, row[1::2])).split(',')
        rows.append(','.join(map(str, list(filter(None, org_row)))))
    return rows

In [ ]:
def ent(rows):
    final_entities_list=[]
    for i in rows:
        entities=[]
        document = nlp(i)
        for k in document.ents:
            entities.append(k)
        final_entities_list.append(entities)
    return final_entities_list

In [ ]:
entities=cleaning(df)

In [ ]:
entities

In [ ]:
comparative_df=df
comparative_df.rename(columns={'Column10':'old'},inplace = True)
entities_df=pd.DataFrame({'result':[','.join(map(str, entities[i].split(','))) for i in range(len(entities))]})
comparative_df=comparative_df.join(entities_df)  

In [ ]:
entities_df

In [ ]:
comparative_df

In [ ]:
entities

In [ ]:
nlp = spacy.load("en_core_web_sm")
def tags(df):
    all_tags=[]
    for i in df:
        tags=[]
        for doc in nlp.pipe(i.split(','), disable=["tagger", "parser"]):
            pair = [(ent.text, ent.label_) for ent in doc.ents]
            print(pair)
            if len(pair)>0:
                tags.append(pair[0])
        all_tags.append(tags)
    return all_tags

In [ ]:
nlp = spacy.load("en_core_web_sm")
def tags(df):
    all_tags=[]
    for i in range(len(df)):
        tags=[]
        for doc in nlp.pipe(df[i].split(',')):
            pairs = [(ent.text, ent.label_) for ent in doc.ents]
            for j in pairs:
                if len(j)>0:
                    tags.append(j)
        all_tags.append(tags)
    return all_tags

In [ ]:
text="Еђsi, Attila#:#Hungarian Academy of Sciences, Hungarian Natural History Museum, \
Research Group for Paleontology, Ludovika tГ©r 2, Budapest 1083, Hungary ,#;#Butler, Richard J.#:#Bayerische \
Staatssammlung fГјr PalГ¤ontologie und Geologie, Richard-Wagner-StraГџe 10, Munich 80333, Germany \
,#;#Weishampel, David B.#:#Center for Functional Anatomy and Evolution, Johns Hopkins University, \
Baltimore, ..."
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
a=tags(entities)
a

In [ ]:
print(spacy.explain("GPE"))

Countries, cities, states


In [ ]:
all_tags=tags(entities)

In [ ]:
Organizations=pd.DataFrame({'Organizations':\
 [list(set([all_tags[i][j][0] for j in range(len(all_tags[i])) if all_tags[i][j][1]=='ORG']))\
                                             for i in range(len(all_tags))]})

In [ ]:
Organizations

In [ ]:
comparative_df=comparative_df.join(Organizations)  

In [ ]:
comparative_df

In [ ]:
org_dict={}
for i in range(len(Organizations['Organizations'])):
    for j in range(len(Organizations['Organizations'][i])):
        if Organizations['Organizations'][i][j] in org_dict:
            org_dict[Organizations['Organizations'][i][j]]+=1
        else: 
            org_dict[Organizations['Organizations'][i][j]]=1
org_dict

In [55]:
Organizations_num_df = pd.DataFrame.from_dict (org_dict, orient='index'). reset_index()
Organizations_num_df.columns = ['Organizations', 'number']

In [ ]:
org_dict

In [ ]:
Organizations_num_df

In [59]:
Organizations_num_df.sort_values(by='number', ascending=False).head()

,Organizations,number
199,Department of Pathology,319
273,Department of Medicine,306
310,Department of Pediatrics,277
370,Department of Physics,261
18,Department of Chemistry,254
